# CS5661 Homework 2
Kevin Lam (CIN: 303061725) 

### Importing necessary libraries

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd

In [2]:
#To allow images to be shown inside Jupyter
%matplotlib inline

### Loading the dataset and creating a helper function

In [3]:
#Load dataset
img_df = pd.read_csv('label.csv')

# Simple helper function for retrieving file/names
def get_file_path(name_of_file):
    x = 'Digit/' + str(name_of_file) + '.jpg'
    return x

### Create list for column names
Format will be 'Pixel_XY' with X & Y ranging from 1-8 to represent each pixel location in a 8x8 grid

In [4]:
column_list = []
for i in range(1,9): # 1-8 inclusive
    for j in range(1,9): # 1-8 inclusive
        xy = (i*10)+j
        name = 'Pixel_' + str(xy)
        column_list.append(name)

pixel_column_list = column_list.copy() # Keep copy of just the pixel columns for easier referencing in training
column_list.append("Digit")
column_list.append("FileName")


# Create dataframe based on column list
df = pd.DataFrame(columns = [column_list])

df.head()

,Pixel_11,Pixel_12,Pixel_13,Pixel_14,Pixel_15,Pixel_16,Pixel_17,Pixel_18,Pixel_21,Pixel_22,...,Pixel_81,Pixel_82,Pixel_83,Pixel_84,Pixel_85,Pixel_86,Pixel_87,Pixel_88,Digit,FileName


### Iterate through the csv, read and append data to dataframe

In [5]:
df = df[0:0] # Clear dataframe

for index, row in img_df.iterrows():                    # Loop through the csv
    filename = get_file_path(row['name of the file'])    # Get file path for image
    img = mpimg.imread(filename).reshape(-1)             # Read image and collapse into a 1-dimensional array (8x8 -> 1x64)
    data = list(img)
    data.append(row['digit'])         # Include the digit
    data.append(filename)             # and file path
    s = pd.Series(data, index=column_list)
    df = df.append(s, ignore_index=True) # Append to dataframe

df.head()

,Pixel_11,Pixel_12,Pixel_13,Pixel_14,Pixel_15,Pixel_16,Pixel_17,Pixel_18,Pixel_21,Pixel_22,...,Pixel_81,Pixel_82,Pixel_83,Pixel_84,Pixel_85,Pixel_86,Pixel_87,Pixel_88,Digit,FileName
0,0,2,88,217,158,21,0,0,16,0,...,4,3,87,228,182,0,9,2,0,Digit/0.jpg
1,0,0,11,187,210,78,5,0,13,5,...,0,0,0,172,251,157,0,12,1,Digit/1.jpg
2,0,0,5,59,240,199,2,0,5,0,...,0,11,0,54,173,255,138,0,2,Digit/2.jpg
3,6,0,108,255,213,13,10,0,2,131,...,0,0,118,217,226,159,0,3,3,Digit/3.jpg
4,0,6,0,34,160,7,7,0,5,0,...,0,0,2,42,250,51,0,4,4,Digit/4.jpg


### Part B & C - Define feature set and target, and train_test_split

In [6]:
X = df[pixel_column_list]
y = df['Digit']
y = y.astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

X_train.head()

,Pixel_11,Pixel_12,Pixel_13,Pixel_14,Pixel_15,Pixel_16,Pixel_17,Pixel_18,Pixel_21,Pixel_22,...,Pixel_77,Pixel_78,Pixel_81,Pixel_82,Pixel_83,Pixel_84,Pixel_85,Pixel_86,Pixel_87,Pixel_88
1739,2,0,175,176,112,2,0,6,7,54,...,21,1,0,1,106,195,124,0,2,0
481,5,0,0,84,240,125,0,6,0,0,...,159,0,0,0,8,122,221,255,131,12
541,0,33,204,226,252,244,117,0,18,31,...,0,0,0,22,165,255,207,10,4,3
332,8,0,53,213,235,141,10,0,0,70,...,170,0,0,2,27,220,254,239,98,0
1705,0,0,116,224,183,0,0,4,0,30,...,48,4,0,6,108,234,255,248,59,1


### Part D - Design and Train ANN

In [7]:
# 1 Hidden Layer with 80 neurons:
my_ANN = MLPClassifier(hidden_layer_sizes=(80,),activation='logistic', 
                       solver='adam', alpha=1e-5, random_state=1, learning_rate_init = 0.002)


my_ANN.fit(X_train, y_train)

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(80,), learning_rate='constant',
       learning_rate_init=0.002, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

### Testing ANN on Training Set | Accuracy & Confusion Matrix

In [8]:
y_predict = my_ANN.predict(X_test)

score = accuracy_score(y_test, y_predict)
print("Accuracy score:",score)

Accuracy score: 0.905555555556


In [9]:
confusion_matrix(y_test, y_predict)

array([[13,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0, 20,  0,  0,  0,  0,  0,  0,  1,  1],
       [ 0,  0, 14,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 20,  0,  0,  0,  2,  0,  0],
       [ 0,  0,  0,  0, 16,  0,  0,  1,  3,  1],
       [ 0,  0,  0,  0,  0, 16,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0, 17,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 17,  1,  0],
       [ 0,  2,  0,  0,  0,  0,  0,  0, 16,  1],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  1, 14]])

### Part E - Using GridSearchCV

In [ ]:
# define a range for the "number of neurons" in the hidden layer for a network with 1 hidden layer:
neuron_number = [(i,) for i in range(50,200)]

# create a dictionary for grid parameter:
param_grid = dict(hidden_layer_sizes = neuron_number)
#print(param_grid,'\n')

# instantiate the model:
my_ANN = MLPClassifier(activation='logistic', solver='adam', 
                                         alpha=1e-5, random_state=1, 
                                           learning_rate_init = 0.002)

# create the grid, and define the metric for evaluating the model: 
grid = GridSearchCV(my_ANN, param_grid, cv=10, scoring='accuracy')

# fit the grid (start the grid search):
grid.fit(X, y)

In [ ]:
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
print(5)